In [92]:
%load_ext autoreload
%autoreload 2

In [116]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [146]:
train_data, test_data = tf.keras.datasets.mnist.load_data(path="mnist.npz")
x_train = train_data[0].reshape(-1,784).astype(float) / 255
y_train = train_data[1]
x_test = test_data[0].reshape(-1,784).astype(float) / 255
y_test = test_data[1]

In [147]:
np.random.seed(42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, shuffle=True)
# y_train, y_val = train_test_split(y_train, test_size=0.1, shuffle=True)


In [148]:
print(len(x_train), len(x_val), len(x_test))

54000 6000 10000


In [149]:
# source: https://keras.io/guides/training_with_built_in_methods/

inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(32, activation="relu", name="dense_1")(inputs)
x = layers.Dense(32, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, activation=None, name="predictions")(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#def loss(true, pred):
#    val = 0.
#    for i in range(len(true)):
#        val -= np.log(pred[i, true[i]])
#    return val / len(true)
model.summary()

Model: "mnist_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 digits (InputLayer)         [(None, 784)]             0         
                                                                 
 dense_1 (Dense)             (None, 32)                25120     
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 predictions (Dense)         (None, 10)                330       
                                                                 
Total params: 26506 (103.54 KB)
Trainable params: 26506 (103.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [150]:
# untrained model
logits = model(x_train)
np.argmax(logits, 1)[0:100]

array([0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 6, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0])

In [151]:
# what we would like to get
y_train[0:100]

array([1, 6, 6, 7, 8, 4, 2, 7, 7, 6, 7, 3, 6, 9, 6, 1, 2, 1, 1, 0, 6, 3,
       0, 5, 3, 9, 2, 4, 1, 3, 3, 8, 7, 2, 0, 8, 5, 2, 5, 5, 6, 1, 1, 2,
       2, 7, 6, 6, 4, 2, 1, 1, 7, 2, 0, 4, 9, 4, 6, 5, 9, 2, 3, 0, 0, 0,
       6, 4, 1, 6, 2, 9, 4, 3, 3, 9, 7, 8, 9, 5, 3, 4, 4, 6, 1, 2, 9, 4,
       5, 6, 3, 1, 9, 4, 0, 9, 0, 3, 3, 4], dtype=uint8)

In [152]:
## Computing a stochastic gradient

# Source: https://keras.io/guides/writing_a_training_loop_from_scratch/

# Example of gradient computation using automatic differentiation

x_train_batch = x_train[:8]
y_train_batch = y_train[:8]
with tf.GradientTape() as tape:
    # Run the forward pass of the layer.
    # The operations that the layer applies
    # to its inputs are going to be recorded
    # on the GradientTape.
    logits = model(x_train_batch)  # Logits for this minibatch

    # Compute the loss value for this minibatch.
    loss_value = loss(y_train_batch, logits)

# Use the gradient tape to automatically retrieve
# the gradients of the trainable variables with respect to the loss.
grad = tape.gradient(loss_value, model.trainable_weights)

## Updating weights

def update_weights(model, update):
    new_weights = model.trainable_weights.copy()
    for i in range(len(new_weights)):
        new_weights[i] = new_weights[i] + update[i]
    model.set_weights(new_weights)


## Evaluate model
samples_for_evaluation = np.random.randint(0, 10000, 1000)
logits = model(x_train[samples_for_evaluation])
loss(y_train[samples_for_evaluation], logits).numpy()
    

2.3407905

In [153]:
from tqdm import tqdm

In [154]:
alpha = 1e-3

In [155]:
def loop(x_train,y_train, epochs=1000, alpha=1e-3, batch_size=128):
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        train_loss = 0
        for i in tqdm(range(0, x_train.shape[0], batch_size)):
            x_train_batch = x_train[i:i+batch_size]
            y_train_batch = y_train[i:i+batch_size]
            with tf.GradientTape() as tape:
            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
                logits = model(x_train_batch)  # Logits for this minibatch

                # Compute the loss value for this minibatch.
                loss_value = loss(y_train_batch, logits)

            # Use the gradient tape to automatically retrieve
            # the gradients of the trainable variables with respect to the loss.
            grad = tape.gradient(loss_value, model.trainable_weights)
            update_weights(model,  [-alpha * g for g in grad])
            train_loss += loss_value.numpy() * len(x_train_batch)
        

        logits = model(x_val)
        val_loss = loss(y_val, logits).numpy()
        train_losses.append(train_losses)
        val_losses.append(val_loss)
        print(f"Epoch {epoch}: train_loss = {train_loss / len(x_train)}, val_loss = {val_loss}")

    return train_losses, val_losses

In [156]:
loop(x_train, y_train, alpha=1e-1)

  0%|          | 0/422 [00:00<?, ?it/s]

100%|██████████| 422/422 [00:03<00:00, 134.61it/s]


Epoch 0: train_loss = 0.5576892976672561, val_loss = 0.2911528944969177


100%|██████████| 422/422 [00:03<00:00, 138.56it/s]


Epoch 1: train_loss = 0.2636889814933141, val_loss = 0.22873498499393463


...

Epoch 64: train_loss = 0.007712439779882078, val_loss = 0.15786884725093842


100%|██████████| 422/422 [00:05<00:00, 81.33it/s] 


Epoch 65: train_loss = 0.007354120419257217, val_loss = 0.1585610955953598


  6%|▋         | 27/422 [00:01<00:19, 20.57it/s] 


KeyboardInterrupt: 

In [ ]:
## Evaluate model
samples_for_evaluation = np.random.randint(0, 10000, 1000)
logits = model(x_train[samples_for_evaluation])
loss(y_train[samples_for_evaluation], logits).numpy()
    

2.053952